In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [3]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [4]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [5]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1262.0


In [6]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  335.0


In [7]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [8]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2993 personas en los últimos 7 días 

Un positivo PCR cada 1381 personas en los últimos 14 días


In [9]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [10]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [11]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [12]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [13]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PCR'] > 0].sort_values('Diferencia PCR', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Huelva,1262.0,183.0,370.0,510743.0,35.830153,72.443479,44.0
Huelva-Costa,684.0,104.0,231.0,288115.0,36.096697,80.176318,26.0
Condado-Campiña,385.0,63.0,104.0,155057.0,40.630220,67.072109,18.0
Huelva (capital),335.0,48.0,104.0,143663.0,33.411526,72.391639,13.0
Bollullos Par del Condado,62.0,21.0,31.0,14272.0,147.141256,217.208520,7.0
Ayamonte,46.0,12.0,29.0,20946.0,57.290175,138.451256,5.0
Aljaraque,69.0,5.0,21.0,21260.0,23.518344,98.777046,3.0
Moguer,35.0,8.0,9.0,22088.0,36.218761,40.746106,3.0
Almonte,53.0,7.0,12.0,24191.0,28.936381,49.605225,2.0


In [14]:
if tabla[tabla['Diferencia PCR'] > 0]['Diferencia PCR'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [15]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Almendro (El),4.0,3.0,3.0,826.0,363.196126,363.196126,0.0
Bollullos Par del Condado,62.0,21.0,31.0,14272.0,147.141256,217.208520,7.0
Villalba del Alcor,42.0,1.0,7.0,3338.0,29.958059,209.706411,0.0
Aroche,8.0,4.0,6.0,3073.0,130.165962,195.248942,0.0
Nerva,23.0,6.0,9.0,5235.0,114.613181,171.919771,0.0
Zalamea la Real,20.0,1.0,5.0,3068.0,32.594524,162.972621,0.0
Manzanilla,7.0,3.0,3.0,2135.0,140.515222,140.515222,NaN
Ayamonte,46.0,12.0,29.0,20946.0,57.290175,138.451256,5.0
Palos de la Frontera,26.0,4.0,15.0,11289.0,35.432722,132.872708,0.0
